Data cleaning

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
df = pd.read_csv(r'C:\Users\AKKODIS\Desktop\venv\code\sales_data.csv')

In [2]:
df1 = df.copy()
display(df1)

,OrderID,CustomerName,Product,Quantity,Price,OrderDate,ShippingCost,City
0,101.0,Alice,iPhone,2,$999.99,2023-01-01,$10,New York
1,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york
2,103.0,NaN,IPHONE,two,-$899.00,"March 5, 2023",Free,NEW YORK
3,104.0,CHARLIE,iPhone,4,"$1,000",2023-03-15,NaN,Los Angeles
4,NaN,dan,Laptop,NaN,$799,2023/04/01,$20,los angeles
5,106.0,Eve,ipad,5,$499.50,05-05-2023,$5.50,Chicago
6,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york


Cleared the garbage values

In [3]:

df1.replace(["?", -1, -1.0, "Not Available", "", "NaN"], value=np.nan, inplace=True)
display(df1)

,OrderID,CustomerName,Product,Quantity,Price,OrderDate,ShippingCost,City
0,101.0,Alice,iPhone,2,$999.99,2023-01-01,$10,New York
1,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york
2,103.0,NaN,IPHONE,two,-$899.00,"March 5, 2023",Free,NEW YORK
3,104.0,CHARLIE,iPhone,4,"$1,000",2023-03-15,NaN,Los Angeles
4,NaN,dan,Laptop,NaN,$799,2023/04/01,$20,los angeles
5,106.0,Eve,ipad,5,$499.50,05-05-2023,$5.50,Chicago
6,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york


Adding missing OrderID

In [4]:
df1['OrderID'] = np.arange(101, 101+len(df))
display(df1['OrderID'])
#alternate method

#df.loc[0, 'OrderID'] = 101
#x = df.loc[0, 'OrderID']
#for i in range(len(df['OrderID'])):
    
    #df.loc[i, 'OrderID'] = x+i
#display(df)   


0    101
1    102
2    103
3    104
4    105
5    106
6    107
Name: OrderID, dtype: int64

Proper capitalization of names and cities; Filling missing values

In [5]:
df1[['CustomerName', 'City']] = df1[['CustomerName', 'City']].apply(lambda col: col.str.title())

df1.fillna({'CustomerName': df1['CustomerName'].mode()[0], 'City': df1['City'].mode()[0]}, inplace=True)

display(df1[['CustomerName', 'City']])

,CustomerName,City
0,Alice,New York
1,Bob,New York
2,Bob,New York
3,Charlie,Los Angeles
4,Dan,Los Angeles
5,Eve,Chicago
6,Bob,New York


Clearing Product(correct capitalization and spelling)

In [6]:
for i in range(len(df1)):
    product = df1.loc[i, 'Product']
    product = product.strip().lower()

    if product[0:3] =='iph':
        product = 'iPhone'
        df1.loc[i, 'Product'] = product
    
    elif product[0:3] =='ipa':
        product = 'iPad'
        df1.loc[i, 'Product'] = product

    elif product[0:3] =='lap':
        product = 'Laptop'
        df1.loc[i, 'Product'] = product
    
display(df1['Product'])

0    iPhone
1    iPhone
2    iPhone
3    iPhone
4    Laptop
5      iPad
6    iPhone
Name: Product, dtype: object

Cleaning Quantity

In [7]:
df1['Quantity'] = pd.to_numeric(df1['Quantity'], errors='coerce').astype('Int64')
print(df1['Quantity'].dtype)

#df1['Quantity'].fillna(df1['Quantity'].median(), inplace=True) this way is not preferred
df1.fillna({'Quantity': df1['Quantity'].median()}, inplace=True)

print(df1['Quantity'])

Int64
0    2
1    3
2    3
3    4
4    3
5    5
6    3
Name: Quantity, dtype: Int64


Clearing Price & ShippingCost
    (Converting Price to string and then replacing $ , and - sign and then converting the string to float
    Also, Changing Free to 0)

In [8]:
df1[['Price', 'ShippingCost']] = df1[['Price', 'ShippingCost']].astype(str)#Converting Float values to string


'''loop for changing free with 0
for i in range(len(df['ShippingCost'])):
    if df1.loc[i, 'Price', 'ShippingCost'] == 'Free':
        df1.loc[i, 'Price', 'ShippingCost'] = '0'
'''


#setting {r"[$,\- ]": "", 'Free': 0}
df1[['Price', 'ShippingCost']] = df1[['Price', 'ShippingCost']].replace({r"[$,\- ]": "", 'Free': 0}, regex=True)

#df1[['Price', 'ShippingCost']] = df1[['Price', 'ShippingCost']].replace(r"[\$ , \-]", "", regex=True) this is without free
#df1['Price'] = df1['Price'].str.replace(",", "", regex=False)#used str.replace because we are doin cheractor/string level replcaement

#Converting string to Float
df1[['Price', 'ShippingCost']] = df1[['Price', 'ShippingCost']].apply(pd.to_numeric, errors='coerce')


#Filling for empty rows
df1.fillna({'Price': df1['Price'].mean(), 'ShippingCost': df1['ShippingCost'].mean()}, inplace=True)

df1[['Price', 'ShippingCost']] = df1[['Price', 'ShippingCost']].round(2)

#cahnging column name
df1.rename(columns={'Price': 'Price($)'}, inplace=True)
df1.rename(columns={'ShippingCost': 'ShippingCost($)'}, inplace=True)
print(df1[['Price($)', 'ShippingCost($)']] )
display(df1)

   Price($)  ShippingCost($)
0    999.99            10.00
1   1099.99            15.00
2    899.00             0.00
3   1000.00            10.92
4    799.00            20.00
5    499.50             5.50
6   1099.99            15.00


,OrderID,CustomerName,Product,Quantity,Price($),OrderDate,ShippingCost($),City
0,101,Alice,iPhone,2,999.99,2023-01-01,10.00,New York
1,102,Bob,iPhone,3,1099.99,01/02/2023,15.00,New York
2,103,Bob,iPhone,3,899.00,"March 5, 2023",0.00,New York
3,104,Charlie,iPhone,4,1000.00,2023-03-15,10.92,Los Angeles
4,105,Dan,Laptop,3,799.00,2023/04/01,20.00,Los Angeles
5,106,Eve,iPad,5,499.50,05-05-2023,5.50,Chicago
6,107,Bob,iPhone,3,1099.99,01/02/2023,15.00,New York


Cleaning OrderDate

In [9]:
df1['OrderDate'] = df1['OrderDate'].apply(pd.to_datetime, errors='coerce')#used .apply so that it catches all the valid Date patterns in Column

display(df1['OrderDate'])

0   2023-01-01
1   2023-01-02
2   2023-03-05
3   2023-03-15
4   2023-04-01
5   2023-05-05
6   2023-01-02
Name: OrderDate, dtype: datetime64[ns]

Final Output

In [10]:
display('Input', df)
print(df1.info())
print('---------------------------------------------------------')
print(df1.describe(include='all'))

df1.to_csv('cleaned_sales.csv', index=False)

'Input'

,OrderID,CustomerName,Product,Quantity,Price,OrderDate,ShippingCost,City
0,101.0,Alice,iPhone,2,$999.99,2023-01-01,$10,New York
1,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york
2,103.0,NaN,IPHONE,two,-$899.00,"March 5, 2023",Free,NEW YORK
3,104.0,CHARLIE,iPhone,4,"$1,000",2023-03-15,NaN,Los Angeles
4,NaN,dan,Laptop,NaN,$799,2023/04/01,$20,los angeles
5,106.0,Eve,ipad,5,$499.50,05-05-2023,$5.50,Chicago
6,102.0,bob,iphon,3,"$1,099.99",01/02/2023,$15,new york


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   OrderID          7 non-null      int64         
 1   CustomerName     7 non-null      object        
 2   Product          7 non-null      object        
 3   Quantity         7 non-null      Int64         
 4   Price($)         7 non-null      float64       
 5   OrderDate        7 non-null      datetime64[ns]
 6   ShippingCost($)  7 non-null      float64       
 7   City             7 non-null      object        
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 587.0+ bytes
None
---------------------------------------------------------
           OrderID CustomerName Product  Quantity     Price($)  \
count     7.000000            7       7       7.0     7.000000   
unique         NaN            5       3      <NA>          NaN   
top          